In [1]:
%load_ext autoreload
import pandas as pd
import numpy as np
import os
import sys


sys.path.append('/Users/vlad/Documents/Code/York/iNet_v2/src/')

from NetworkAnalysis.ExperimentSet import ExperimentSet
from NetworkAnalysis.GraphToolExp import GraphToolExperiment as GtExp


# Gsea libraries
import matplotlib.pyplot as plt

import graph_tool.all as gt

# %matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
results_path = "../../results/exp/"
data_base = "../../data/"
base_path = "../../results/network_II/"

figures_path = "../network_II/reward/"

# base_sbm_path = "../../../iNet_v2/"
sbm_exps_path = "sbm/"
control_exps_path = "mut_control_v3/"

vu_output = pd.read_csv(f"{data_base}/metadata/VU_clustering_v3.tsv", sep="\t", index_col="Sample")

tcga_mutations_df = pd.read_csv(f"{data_base}/tumour/mutations_tcga.csv")
tcga_mutations_df = tcga_mutations_df[tcga_mutations_df["count"] != 0].set_index("gene")

all_tum_tpms = pd.read_csv(f"{data_base}/tumour/tum_TPMs_selected_genes_gc42_all_v4.tsv", sep="\t", index_col="genes")


healthy_metadata = pd.read_csv(f"{data_base}/metadata/healthy_bladder_metadata.tsv", sep="\t", index_col="Sample")

# tf list
tf_path = f"{data_base}/metadata/TF_names_v_1.01.txt"
if os.path.exists(tf_path):
    tf_list = np.genfromtxt(fname=tf_path, delimiter="\t", skip_header=1, dtype="str")

In [3]:
hsbm_v3 = ExperimentSet("v3", base_path=base_path, exp_path=sbm_exps_path, mut_df=tcga_mutations_df, sel_sets=None, exp_type="iNet")

exps = {}
for idx, exp in enumerate(hsbm_v3.get_exps()):
    if exp.sbm_method != "hsbm":  # or idx != 2:
        continue

    if "standard" in exp.name:
        continue

    print(f"Loading Graph-Tool for {exp.type}")
    exp = GtExp.from_pgcna_inet(exp, rel_path="")

    # partitions from hSBM
    exp.hsbm_add_vp(mut_df=tcga_mutations_df)
    com_df, _ = exp.hsbm_get_gt_df()

    exp.export_to_gephi(save=False, com_df=com_df)

    # Need it for comparison
    exps[exp.type] = exp

hsbm_v3.exps = exps

##### Experiment labels:  dict_keys(['standard_5K_6TF_sbm', 'standard_5K_6TF_hsbm', 'sigmoid_5K_6TF_hsbm'])
Loading Graph-Tool for sigmoid_5K_6TF_hsbm


In [4]:
%autoreload 2
sel_gt: GtExp = hsbm_v3.exps["sigmoid_5K_6TF_hsbm"]
gt_state: gt.NestedBlockState = sel_gt.hstateObj["state"]

sel_gt.hsbm_add_vp(mut_df=tcga_mutations_df)

gt_g = sel_gt.gt_g
pv = sel_gt.hstateObj["pv"]
pmode = sel_gt.hstateObj["pmode"]

com_df, _ = sel_gt.hsbm_get_gt_df()
com_df["max_b"] = com_df["P_lvl_0"]

gt.remove_self_loops(gt_g)

######## ModCon and MEVS ########
sel_gt.gt_modCon_MEV(all_tpms=sel_gt.tpm_df)

# Compute and store the nodes/edges with all the information
sel_gt.export_to_gephi(save=False, com_df = com_df)
sel_gt.add_gt_prop_draw(gt_g, com_df=com_df, tf_list=tf_list)
graph_stats = sel_gt.compute_graph_stats()


# Visualisaition

In [5]:
sel_gt: GtExp = hsbm_v3.exps["sigmoid_5K_6TF_hsbm"]
gt_g: gt.Graph = sel_gt.gt_g

com_df, _ = sel_gt.hsbm_get_gt_df()
sel_gt.graph_type = "hsbm"
sel_gt.nodes_df["node_idx"] = com_df["node_idx"]
sel_gt.nodes_df["max_b"] = com_df["max_b"]

sel_gt.hsbm_add_vp(mut_df=tcga_mutations_df)
sel_gt.add_gt_prop_draw(sel_gt.gt_g, com_df=com_df, tf_list=tf_list)

gt_state: gt.NestedBlockState = sel_gt.hstateObj["state"]
pv = sel_gt.hstateObj["pv"]

vp_dg = gt_g.degree_property_map(deg="total", weight=gt_g.ep["weight"])

In [13]:
draw_results = gt_state.draw(
    # pos=pos,
    layout="sfdp",
    # vertex_shape="pie",
    # vertex_pie_fractions=pv,
    inline=False,
    # vertex_color=gt_g.vp["colors_rank"],
    # vertex_color=gt.prop_to_size(gt_g.vp["modCon_rank"], 0, 100, power=0.1),
    vertex_text=gt_g.vp.prcsd_gene,
    display_props=gt_g.vp.display_prop,
    subsample_edgesint=int(gt_g.num_edges() * 0.3),
    output_size=(2000, 2000),  # good to visualised in Notebook
    # ######## For mut_count ########
    # vertex_size=gt.prop_to_size((gt_g.vp["sel_com"]), 1, 10, power=0.5),
    # vertex_font_size=gt.prop_to_size(gt_g.vp.is_tf, 15, 30, power=0.5),
    # edge_pen_width=gt.prop_to_size(gt_g.ep.weight, 2, 10, power=1),
    # vertex_font_size=gt.prop_to_size(gt_g.vp.mut_count, 4, 10, power=1.0),  # 5, 30 for notebook  viz
    ##### edges properties
    # output=f"{figures_path}/{sel_gt.name}_3_6.pdf",
    # hide=0,
    update_layout=False,
    nodesfirst=False,
    # beta=0.7,
    # chord_scale=5,
    display_props_size=16,
)